In [2]:
import pandas as pd
from haversine import haversine
import numpy as np

# 데이터 불러오기

In [4]:
# DB 연결
from DB import *
db_connection = connect_db()

## 메인데이터 xy좌표 불러오기 (main)

In [5]:
main = load_data_from_rds('생태통로', db_connection)

## 교통량 데이터 불러오기 (traffic)

In [6]:
traffic= pd.read_excel('.././data/variables/2021년 도로 종류별 교통량 및 X.Y좌표.xlsx', sheet_name = 'X.Y좌표')

## 환경성 영향평가 데이터 불러오기 (env) 

In [7]:
env = pd.read_csv('.././data/variables/국토환경성 평가지도 병합.csv', encoding = 'cp949')

## 건물까지거리 데이터 불러오기 (build)

In [8]:
build = pd.read_csv(".././data/variables/건물까지거리_병합.csv", encoding = 'cp949')

# 1. 교통량 전처리

## 7.8km 이내에 교통량 찾기 

In [15]:
count = 0
index = []
for i in range(len(main)):
    for j in range(len(traffic)):
        dist = haversine((main[:]["위도"][i],main[:]["경도"][i]), (traffic[:]["XCODE"][j],traffic[:]["YCODE"][j]), unit = 'km')
        count +=1
        if dist <= 7.82 : #각 구간간 거리평균이 7.82
            index.append([i,j]) # 7.8km 이내에 있는 교통량의 인덱스 리스트에 넣기
            if index[-1][0] in[100,200,300,400,500]: print(index[-1][0])

100
100
100
100
100
100
200
200
200
200
200
200
200
200
200
200
200
200
200
200
300
300
300
300
400
400
400
400
400
400
400


In [16]:
# 리스트 데이터 프레임으로 만들기
index2 = pd.DataFrame(index) # 0 : 생태통로 index, 1: 교통량 index
index2

,0,1
0,0,1180
1,0,1182
2,0,1183
3,0,1649
4,0,1651
...,...,...
4338,496,1743
4339,496,2535
4340,496,2538
4341,496,2590


In [17]:
# 교통량 값만 불러오기
AADT=pd.DataFrame(traffic["AADT"].iloc[index2[1]])
AADT= AADT.reset_index()

In [19]:
# 교통량 값이랑 인덱스 데이터 프레임이랑 합치기 
index3 = pd.concat([index2, AADT],axis =1)

# 하나의 생태통로에 여러개의 교통량이 관측되면 평균값으로 계산
index4 = index3[[0,"AADT"]].groupby(0).mean()
index4

,AADT
0,
0,7493.875000
1,3573.857143
2,3430.428571
3,3430.428571
4,34270.190476
...,...
492,34270.190476
493,34542.400000
494,34270.190476


In [20]:
prepro_1 = pd.concat([main['번호'], index4], axis = 1)
prepro_1

,번호,AADT
0,1495,7493.875000
1,1277,3573.857143
2,1278,3430.428571
3,1279,3430.428571
4,1090,34270.190476
...,...,...
492,1089,34270.190476
493,1087,34542.400000
494,1088,34270.190476
495,1091,26748.437500


# 2. 환경 영향성 평가 전처리

In [25]:
prepro_2 = pd.merge(prepro_1, env, left_on = "번호", right_on = "번호", how = "inner")[['번호','AADT','SAMPLE_1']]

In [30]:
prepro_2

,번호,AADT,SAMPLE_1,distance_m
0,1495,7493.875000,2,0.196597
1,1277,3573.857143,1,0.358139
2,1278,3430.428571,2,0.139054
3,1279,3430.428571,2,0.044559
4,1090,34270.190476,5,0.172184
...,...,...,...,...
492,1089,34270.190476,5,0.179017
493,1087,34542.400000,5,0.094304
494,1088,34270.190476,5,0.124789
495,1091,26748.437500,2,0.070303


# 3. 건물까지 거리 전처리

In [28]:
# m 단위 km 단위로 바꾸기
build['distance_m'] = build['distance']/1000

In [29]:
prepro_3 = pd.merge(prepro_2, build, left_on = "번호", right_on = "번호", how = "inner")[['번호','AADT', 'SAMPLE_1', 'distance_km']]

# 열 이름 바꾸기

In [31]:
prepro_3.columns = ['번호', '교통량', '환경영향평가점수', '건물까지거리(km)']
prepro_3

,번호,교통량,환경영향평가점수,건물까지거리(km)
0,1495,7493.875000,2,0.196597
1,1277,3573.857143,1,0.358139
2,1278,3430.428571,2,0.139054
3,1279,3430.428571,2,0.044559
4,1090,34270.190476,5,0.172184
...,...,...,...,...
492,1089,34270.190476,5,0.179017
493,1087,34542.400000,5,0.094304
494,1088,34270.190476,5,0.124789
495,1091,26748.437500,2,0.070303


# 데이터 베이스에 올리기

## 변수 데이터 불러오기

In [33]:
variables = load_data_from_rds('변수관계설정', db_connection)

,번호,등산로까지 최단거리(km),식생,경사도,지형기호(2.3km),주변 동물종 개수,주변동물 출현빈도,주변 로드킬 빈도,산책로까지의 최단 거리(km),최고제한속도(km/h),농가까지의 거리(km)
0,1001,1.903039,0,6,X,NaN,NaN,0,237.040073,80.0,0.125686
1,1002,0.120319,1,5,X,4.0,4.0,0,122.055124,60.0,0.074627
2,1004,0.047211,1,2,산지,1.0,1.0,0,125.516218,60.0,0.110617
3,1005,0.233820,1,2,X,4.0,4.0,0,128.701343,80.0,0.009030
4,1006,2.172428,0,4,하천,3.0,3.0,0,103.572133,80.0,0.098269
...,...,...,...,...,...,...,...,...,...,...,...
492,1561,0.000416,1,2,산지,3.0,5.0,4,219.634740,60.0,0.087448
493,1562,2.835246,1,5,산지,6.0,27.0,0,38.432601,80.0,0.137822
494,1564,0.886170,1,4,X,1.0,1.0,0,248.086515,80.0,0.054073
495,1567,9.125829,1,5,하천,4.0,5.0,0,283.374921,60.0,0.102655


In [38]:
# 변수 데이터랑 합치기
final = pd.merge(variables , prepro_3, on = '번호')

In [ ]:
upload_data_to_rds(final, '변수관계설정', db_connection)

In [40]:
# DB에 올리기
# final.to_sql(name = '변수관계설정', con=db_connection, if_exists='replace', index=False)